<a href="https://colab.research.google.com/github/Yashwanth-1406/Pyspark-yashwanth/blob/main/Advance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **adv.pyspark(window function)**

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.appName("AdvancedOps").getOrCreate()

data = [
    (1, "Alice", 2000, ["math", "science"], {"city": "NYC", "zip": "10001"}),
    (2, "Bob", 1500, ["english"], {"city": "SF", "zip": "94105"}),
    (3, "Charlie", 2200, ["math", "history", "science"], {"city": "NYC", "zip": "10001"}),
    (4, "David", 1200, ["art"], {"city": "LA", "zip": "90001"}),
]

df = spark.createDataFrame(data, schema=["id", "name", "salary", "subjects", "address"])
df.show(truncate=False)

+---+-------+------+------------------------+---------------------------+
|id |name   |salary|subjects                |address                    |
+---+-------+------+------------------------+---------------------------+
|1  |Alice  |2000  |[math, science]         |{zip -> 10001, city -> NYC}|
|2  |Bob    |1500  |[english]               |{zip -> 94105, city -> SF} |
|3  |Charlie|2200  |[math, history, science]|{zip -> 10001, city -> NYC}|
|4  |David  |1200  |[art]                   |{zip -> 90001, city -> LA} |
+---+-------+------+------------------------+---------------------------+



In [6]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank

# Define window spec
window_spec = Window.partitionBy("address.city").orderBy("salary")

# Apply rank over the window
df.withColumn("rank", rank().over(window_spec)).show()


+---+-------+------+--------------------+--------------------+----+
| id|   name|salary|            subjects|             address|rank|
+---+-------+------+--------------------+--------------------+----+
|  4|  David|  1200|               [art]|{zip -> 90001, ci...|   1|
|  1|  Alice|  2000|     [math, science]|{zip -> 10001, ci...|   1|
|  3|Charlie|  2200|[math, history, s...|{zip -> 10001, ci...|   2|
|  2|    Bob|  1500|           [english]|{zip -> 94105, ci...|   1|
+---+-------+------+--------------------+--------------------+----+



In [7]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number, rank, dense_rank, max, sum, avg

# Employee Data
data = [
    (1, "John", "Sales", 3000),
    (2, "Jane", "Finance", 4000),
    (3, "Mike", "Sales", 3500),
    (4, "Alice", "Finance", 3800),
    (5, "Bob", "IT", 4500),
    (6, "Tom", "Sales", 3700),
    (7, "Jerry", "Finance", 4200),
    (8, "Sam", "IT", 4700),
    (9, "Steve", "Sales", 3100),
    (10, "Rachel", "IT", 4600)
]
columns = ["EmpID", "Name", "Department", "Salary"]
df = spark.createDataFrame(data, columns)

df.show()

+-----+------+----------+------+
|EmpID|  Name|Department|Salary|
+-----+------+----------+------+
|    1|  John|     Sales|  3000|
|    2|  Jane|   Finance|  4000|
|    3|  Mike|     Sales|  3500|
|    4| Alice|   Finance|  3800|
|    5|   Bob|        IT|  4500|
|    6|   Tom|     Sales|  3700|
|    7| Jerry|   Finance|  4200|
|    8|   Sam|        IT|  4700|
|    9| Steve|     Sales|  3100|
|   10|Rachel|        IT|  4600|
+-----+------+----------+------+



In [8]:
windowSpec=Window.partitionBy("Department").orderBy(col("Salary").desc())
df.withColumn("Rank",rank().over(windowSpec)).show()

+-----+------+----------+------+----+
|EmpID|  Name|Department|Salary|Rank|
+-----+------+----------+------+----+
|    7| Jerry|   Finance|  4200|   1|
|    2|  Jane|   Finance|  4000|   2|
|    4| Alice|   Finance|  3800|   3|
|    8|   Sam|        IT|  4700|   1|
|   10|Rachel|        IT|  4600|   2|
|    5|   Bob|        IT|  4500|   3|
|    6|   Tom|     Sales|  3700|   1|
|    3|  Mike|     Sales|  3500|   2|
|    9| Steve|     Sales|  3100|   3|
|    1|  John|     Sales|  3000|   4|
+-----+------+----------+------+----+



In [9]:
windowSpec=Window.partitionBy("Department")
df.withColumn("MaxSalary",max("Salary").over(windowSpec)).show()

+-----+------+----------+------+---------+
|EmpID|  Name|Department|Salary|MaxSalary|
+-----+------+----------+------+---------+
|    2|  Jane|   Finance|  4000|     4200|
|    4| Alice|   Finance|  3800|     4200|
|    7| Jerry|   Finance|  4200|     4200|
|    5|   Bob|        IT|  4500|     4700|
|    8|   Sam|        IT|  4700|     4700|
|   10|Rachel|        IT|  4600|     4700|
|    1|  John|     Sales|  3000|     3700|
|    3|  Mike|     Sales|  3500|     3700|
|    6|   Tom|     Sales|  3700|     3700|
|    9| Steve|     Sales|  3100|     3700|
+-----+------+----------+------+---------+



In [10]:

windowSpec = Window.partitionBy("Department")
df.groupBy("Department") \
  .agg(max("Salary").alias("MaxSalary")) \
  .show()


+----------+---------+
|Department|MaxSalary|
+----------+---------+
|     Sales|     3700|
|   Finance|     4200|
|        IT|     4700|
+----------+---------+



In [11]:
from pyspark.sql.window import Window
from pyspark.sql.functions import sum

windowSpec = Window.partitionBy("Department") \
                   .orderBy("Salary") \
                   .rowsBetween(Window.unboundedPreceding, 0)

df.withColumn("CumulativeSalary", sum("Salary").over(windowSpec)).show()


+-----+------+----------+------+----------------+
|EmpID|  Name|Department|Salary|CumulativeSalary|
+-----+------+----------+------+----------------+
|    4| Alice|   Finance|  3800|            3800|
|    2|  Jane|   Finance|  4000|            7800|
|    7| Jerry|   Finance|  4200|           12000|
|    5|   Bob|        IT|  4500|            4500|
|   10|Rachel|        IT|  4600|            9100|
|    8|   Sam|        IT|  4700|           13800|
|    1|  John|     Sales|  3000|            3000|
|    9| Steve|     Sales|  3100|            6100|
|    3|  Mike|     Sales|  3500|            9600|
|    6|   Tom|     Sales|  3700|           13300|
+-----+------+----------+------+----------------+



In [12]:
windowSpec = Window.partitionBy("Department") \
                   .orderBy("Salary") \
                   .rowsBetween(-1, 0)

df.withColumn("PreviousSalary", first("Salary").over(windowSpec)).show()

+-----+------+----------+------+--------------+
|EmpID|  Name|Department|Salary|PreviousSalary|
+-----+------+----------+------+--------------+
|    4| Alice|   Finance|  3800|          3800|
|    2|  Jane|   Finance|  4000|          3800|
|    7| Jerry|   Finance|  4200|          4000|
|    5|   Bob|        IT|  4500|          4500|
|   10|Rachel|        IT|  4600|          4500|
|    8|   Sam|        IT|  4700|          4600|
|    1|  John|     Sales|  3000|          3000|
|    9| Steve|     Sales|  3100|          3000|
|    3|  Mike|     Sales|  3500|          3100|
|    6|   Tom|     Sales|  3700|          3500|
+-----+------+----------+------+--------------+

